In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("fc97dd38-631a-4a2e-a126-3f061e243909")

In [ ]:
#Load all relevant libraries

from yields.providers.filesystem import ProviderFactoryFilesystem #To write into filesystem

import plotly.express as px
import ast #to convert the list of vars (which is of string type) to a real list type.

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#seaborn to create pairwise scatterplots
import seaborn as sns
sns.set(style="ticks", color_codes=True)

#some ML libraries
import scipy
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor , NearestNeighbors
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.impute import SimpleImputer

import json, pickle

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder , QuantileTransformer
from sklearn.compose import ColumnTransformer

In [ ]:
sklearn.__version__

In [ ]:
input_artifact_ds = session.io.input('ds')
file_handler = input_artifact_ds.file(input_artifact_ds.files[0]).open()
raw_data = pd.read_csv(file_handler)

In [ ]:
raw_data.head()

In [ ]:
input_artifact_dmm_ds = session.io.input('dmm_ds')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
schema = json.load(file_handler)

In [ ]:
target, continuous_inputs, categorical_inputs = schema['target'],  schema['continuous_inputs'] , schema['categorical_inputs']
inputs = continuous_inputs + categorical_inputs
print('TARGET VARIABLE: ', target)
print('CONTINUOUS inputs: ', continuous_inputs)
print('COUNTABLE inputs: ', categorical_inputs)

In [ ]:
for var in categorical_inputs:
    print(var + ' - number of distinct categories: ' + str(len(raw_data[var].unique())))

List of potential risk drivers (untransformed)

In [ ]:
vars_to_drop = []
no_transform = []


cont_vars = continuous_inputs
vars_to_embed = [] #Should be either string or integer (dtype 'object' or int32)
vars_to_ohe = categorical_inputs #Should be either string or integer (dtype 'object' or int32)
cat_vars = vars_to_ohe + vars_to_embed


# cont_vars = ['Age','Fare']
# vars_to_embed = ['Cabin' , 'SibSp', 'Parch','Pclass'] #Should be either string or integer (dtype 'object' or int32)
# vars_to_ohe = ['Sex','Embarked'] #Should be either string or integer (dtype 'object' or int32)
# cat_vars = vars_to_ohe + vars_to_embed
# target = 'Survived'


print(f"Continuous input variables: {cont_vars}")
print(f"Categorical input variables: {cat_vars}")
print(f"Target to be predicted: {target}")

# EXPLORATORY DATA ANALYSIS

For each categorical potential model variable, let's see how the observed target rate is distributed over the categories.

In [ ]:
# One can visualize the observed target rate for every categorical input variable: this gives insights already insights on how well these variable might be able to predict

target_rate_full_data = raw_data[target].mean()

for cat in cat_vars:
    
    fig = make_subplots(rows = 1,cols = 2, subplot_titles = ("Number of observations" , "Observed target rate"))
    
    tmp = raw_data[[cat,target]]
    # tmp = tmp.dropna() 
    
    #For each categorical input variable: create histogram that shows the number of observations - this might give an indication on whether to group categories
    tmp_grp = tmp.groupby(cat)[target].agg('count')
    
    # fig.update_layout(title_text = 'Observed target rate by ' + cat)
    fig.update_xaxes(title_text = cat)
    fig.add_trace(go.Bar(x=tmp_grp.index , y = tmp_grp.values, showlegend = False) , 1, 1 )
    
    #For each categorical input variable: create histogram that shows observed target rate for each category - this might give an indication on what variables might help to discriminate between target 0 and 1
    tmp_grp = tmp.groupby(cat)[target].agg('mean')
   
    fig.add_trace(go.Bar(x=tmp_grp.index , y = tmp_grp.values, showlegend = False) , 1, 2 )
    fig.add_hline(y=target_rate_full_data, line_width = 5, line_dash = 'dashdot', annotation_text = "Total observed target rate", row = 1, col = 2)
    fig.update_xaxes(title_text = cat)
    fig.update_yaxes(tickformat = ',.1%' , row = 1, col = 2) #observed rate is a percentage, so show it like that (up to 1 decimal number after comma).
    
    
    fig.show()
    

# PREPROCESSING THE DATASET (sklearn)

Below we show how any given tabular dataset could be preprocessed. Several choices need to be made by the modellers, most of which are documented along the code. <br>
The code works as follows:

* Declare all potential model variables as follows (should be of type list)
    * variables you do not want transform at all
    * continuous variables that you might want to scale
    * categorical variables that you want to ordinally encode (either if you think that a) a natural ordering exists between the categories, e.g. cold < warm < hot, or as a step-up to be used inside a so-called embedding layer (the latter is used when lots of categories are present or if you think some relation between the categories of a single variable exist, e.g. zip-code might reveal neigbhouring cities)
    * categorical variables that you want to one-hot-encode: used when number of categories is 'small' and we do not suspects any relationship between the different categories.
    
* Choose a type of imputation for both the categorical and continuous variables, i.e. to replace missing values 
* Choose a type of scaler to transform the continuous variables to make these more or less 'normally distributed' and 'have them inside the range (-1,1)'. This helps the training when fitting a model.

Off course, it is up to the user to create his/her own choices that might be deemed promising. For example, one can declare different scalers for each continuous variable. Also, when does not want to use an embedding layer for a high cardinality (lots of categories) variable one might want to treat this variable in a second step as a continuous variables and scale the categories 1, 2, .... $N_{cat}$ down to range (-1,1).

The above clearly indicates that preprocessing of data is one of many choices to be made to get to a descent model performance.

In [ ]:


raw_model_vars = no_transform + cont_vars + vars_to_embed + vars_to_ohe

data_train = raw_data.copy()
data_train = data_train.drop(vars_to_drop, axis = 1)
data_train = data_train[raw_model_vars + [target]] 


In [ ]:
data_train

In [ ]:
print('Variables to be left untransformed: ' + str(no_transform))
print('Continuous variables to be transformed: ' + str(cont_vars))
print('Categorical variables to be ordinal encoded (embedding): ' + str(vars_to_embed))
print('Categorical variables to be one-hot-encoded: ' + str(vars_to_ohe))


In [ ]:
#Let's create a train and validation set to pass to the fit function of the NN.
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data_train[raw_model_vars], stratify = data_train[target] , test_size = 0.2, random_state=42)

X_train = data_train[raw_model_vars].copy()
X_test = X_train.sample(int(0.2*len(X_train)))

print("# train samples: " + str(len(X_train)) + "\n# validation samples: " + str(len(X_test)))

In [ ]:
display(X_train.head().style.set_caption("Raw (unpreprocessed) data samples"))
display(X_train.isnull().sum())

In [ ]:
"""

PREPROCESSING THE (imputed) DATASET

CONTINOUS VARIABLES

CATEGORICAL VARIABLES:

    + treated as one-hot: ohe
    + to embed: through labelencoder (turn into integers): will be passed to embedding layer inside the NN architecture

"""

# # pipeline for categorical data
ohe_preprocessing = Pipeline(
    [
    ("cat_impute" , SimpleImputer(strategy="most_frequent")),
    ("cat_ohe" , OneHotEncoder(handle_unknown="ignore", sparse=False))
    ]
    )

ord_preprocessing = make_pipeline(
    
    SimpleImputer(strategy="most_frequent"), #impute with the most frequently ocurring category
    OrdinalEncoder(handle_unknown = 'use_encoded_value' , unknown_value = 998 , dtype = "int") #if during transformation (e.g. on another dataset) unknown values seen during 'fit' are encountered, we replace this value with np.nan
    
    )

# # pipeline for numerical data
num_preprocessing = make_pipeline(
    
    SimpleImputer(strategy="median"), #https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute
    StandardScaler() #https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

    )

preprocessing = ColumnTransformer([('categorical_ohe' , ohe_preprocessing, vars_to_ohe) , ('categorical_ordinal' , ord_preprocessing, vars_to_embed) , ('numerical' , num_preprocessing, cont_vars) ] ,  remainder = "passthrough" )

""" FITTING THE PREPROCESSOR """
print("OVERVIEW OF THE PREPROCESSOR OBJECT")
preprocessing.fit(X_train)

In [ ]:
"""" Retrieving the transformed column names

only for the one-hot-encoded variables, the output names is changed and should be made meaningful

Respect order of the pipeline.
"""

ohe_var_name = preprocessing.transformers_[0][1]['cat_ohe'].get_feature_names_out(vars_to_ohe)

transformed_var_names = ohe_var_name.tolist() + vars_to_embed + cont_vars + no_transform

X_train_transformed = pd.DataFrame(data = preprocessing.transform(X_train) , index = X_train.index , columns = transformed_var_names)

In [ ]:
display(X_train_transformed.head(5).style.set_caption("Overview of some preprocessed observations"))
display(X_train_transformed.info())

In [ ]:
"""SHOW THE RAW AND PREPROCESSED CONTINOUS FEATURES"""

for var in cont_vars:
    
    fig = make_subplots(rows = 1,cols = 2, subplot_titles = ("Raw data" , "Preprocessed data"))
    fig.add_trace(go.Histogram(x = X_train[var], showlegend = False) , 1, 1)
    fig.update_xaxes(title_text = var, row = 1, col = 1)
    fig.update_yaxes(title_text = 'Counts' , row = 1, col = 1)
    
    fig.add_trace(go.Histogram(x = X_train_transformed[var], showlegend = False) , 1, 2)
    fig.update_xaxes(title_text = var, row = 1, col = 2)
    fig.update_yaxes(title_text = 'Counts' , row = 1, col = 2)
    fig.show()


Let's finally join the target back onto the preprocessed dataset after which we can start modelling

In [ ]:
X_train_ = X_train_transformed.join(raw_data[[target]])
X_train_.head()

Let us also transform the test set using the trained preprocessor and join the target

In [ ]:
X_test_transformed = pd.DataFrame(data = preprocessing.transform(X_test) , index = X_test.index , columns = transformed_var_names)
X_test_ = X_test_transformed.join(raw_data[[target]])
X_test_.head()

### OPEN QUESTION: handling categories that were not present in the train set.

Different solutions might exist:

* 'spoil' the train set such that it contains some corrupted observations however for which we know all categories are present
* use the built-in preprocesssing layers from which the vocabulary automatically takes into account unknown categories 
* train on the full train set, but this makes impossible to do hyperparameter tuning as we do not have a separate validation set.
* adjust the ordinalencoder such that unseen categories are mapped to some predefined value (either constant, random, or through business insight).


# LET'S CREATE A BENCHMARK MODEL

Obviously, we need to state an objective that determines how good our model is performing. Given a binary target model, we use one of the most simple metrics, the so-called Area-under-the-ROC-curve. It tells us how well the model can discriminate between the target 1 and 0. A ROC of 100% means that the model is able to perfectly discriminate both labels (there are no false-positives neither false negatives). A ROC of 50% on the other hand corresponds to a random guess, so this means the model can be considered useless.

## LOGISTIC REGRESSION

In [ ]:
"""LOGISTIC REGRESSION"""

train_data, train_target = X_train_.drop(target, axis = 1), X_train_[target]

log_reg = sklearn.linear_model.LogisticRegression(penalty='l1', fit_intercept=True, solver='liblinear')

log_reg.fit(train_data, train_target)

trained_log_reg = pd.DataFrame(data = log_reg.coef_.flatten(), index = train_data.columns, columns = ["Fitted coefficients"])
trained_log_reg.loc["Intercept"] = log_reg.intercept_

display(trained_log_reg.style.set_caption("Fitted coefficients of a logistic regression: positive (negative) values means 'higher values for the variables means higher (lower) probability of having target = 1" ))

In [ ]:
#AREA UNDER THE CURVE

fpr, tpr, _ = sklearn.metrics.roc_curve(train_target,  log_reg.predict_proba(train_data)[:,1])


#Test set
test_data, test_target = X_test_.drop(target, axis = 1), X_test_[target]
fpr_t, tpr_t, _ = sklearn.metrics.roc_curve(test_target,  log_reg.predict_proba(test_data)[:,1])

print("AUC train set: " + str(sklearn.metrics.roc_auc_score(train_target, log_reg.predict_proba(train_data)[:,1])))
print("AUC test set: " + str(sklearn.metrics.roc_auc_score(test_target, log_reg.predict_proba(test_data)[:,1])))

fig = make_subplots()
fig.update_layout(title_text = 'ROC-curve')
fig.update_xaxes(title_text = 'TPR')
fig.update_yaxes(title_text = 'FPR')
fig.add_trace(go.Scatter(x = fpr , y = tpr  , name = 'train'))
fig.add_trace(go.Scatter(x = fpr_t , y = tpr_t  , name = 'test'))
fig.add_trace(go.Scatter(x = [0,1] , y = [0,1] , name = 'random' ))
fig.show()



## MAKING A COMPLETE PIPELINE

In the previous cells, we trained separately the preprocessor and the model and save them as separate objects. This has the main drawback that in a downstream instance, the preprocessing step has to be executed as well leading to additional code. This is not a problem by itself if one is interested in the preprocessing analysis. 
However, for the XAI notebook (downstream) it's easier to create and save a full pipeline including both the preprocessor and model training, such that we only have to make predictions using this single pipeline object. We can, in this case, also stick to the untransformed variables as input for the pipeline and we can more easily assess the model's transparency based on the raw variables and not the transformed ones (e.g. gender will be treated as a single feature and not as a binary / dummy variable )

In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessing),
                           ('classifier', LogisticRegression(penalty='l1', fit_intercept=True, solver='liblinear'))])

pipeline.fit(data_train.drop(target,axis=1), data_train[target]) #Use the raw variables and not the transformed ones. The transformation is done in the pipeline itself.

pipeline.predict_proba(data_train.drop(target,axis=1))

In [ ]:
#Enrich the original dataset with the above predictions
raw_data_enriched = raw_data.copy()
raw_data_enriched['prediction'] = pipeline.predict_proba(raw_data[inputs])[:,1]

display(raw_data_enriched.head())


#Save sklearn pipeline
output_artifact = session.io.output('Pipeline_Trained_Logistic_Regression') #use alias as determined per UI to get the correct output artifact.
artifact_output_version = output_artifact.file("sklearn_pipeline.pickle",ProviderFactoryFilesystem()) #.file() takes a name and a provider and it returns a buffer that can be written to.
pickle.dump(pipeline, artifact_output_version)
output_artifact.save()
      
    
artifact = session.io.output('Data_Enriched_with_prediction')
artifact_output_version = artifact.file('prediction_enriched_dataset.csv', ProviderFactoryFilesystem())
raw_data_enriched.to_csv(artifact_output_version, index=False)
artifact.save()

## GRADIENT BOOSTING

In [ ]:
GB = sklearn.ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0)

GB.fit(train_data, train_target)

In [ ]:
"""Feature importance"""

feature_imp = pd.DataFrame(data = GB.feature_importances_, index = train_data.columns, columns = ["Feature importance Gradient Boosting"])
display(feature_imp.sort_values("Feature importance Gradient Boosting", ascending = False).style.set_caption(
    "Feature importance of trained GradientBoostingClassifier. Higher values indicate more important variables in explaining the target.")
       )

In [ ]:
#AREA UNDER THE CURVE

fpr, tpr, _ = sklearn.metrics.roc_curve(train_target,  GB.predict_proba(train_data)[:,1])

#Test set
test_data, test_target = X_test_.drop(target, axis = 1), X_test_[target]
fpr_t, tpr_t, _ = sklearn.metrics.roc_curve(test_target,  GB.predict_proba(test_data)[:,1])

print("AUC train set: " + str(sklearn.metrics.roc_auc_score(train_target, GB.predict_proba(train_data)[:,1])))
print("AUC test set: " + str(sklearn.metrics.roc_auc_score(test_target, GB.predict_proba(test_data)[:,1])))

fig = make_subplots()
fig.update_layout(title_text = 'ROC-curve')
fig.update_xaxes(title_text = 'TPR')
fig.update_yaxes(title_text = 'FPR')
fig.add_trace(go.Scatter(x = fpr , y = tpr  , name = 'train'))
fig.add_trace(go.Scatter(x = fpr_t , y = tpr_t  , name = 'test'))
fig.add_trace(go.Scatter(x = [0,1] , y = [0,1] , name = 'random' ))
fig.show()

In [ ]:

output_artifact = session.io.output('Trained_Preprocessor') #use alias as determined per UI to get the correct output artifact.
artifact_output_version = output_artifact.file("trained_preprocessor.pickle",ProviderFactoryFilesystem()) #.file() takes a name and a provider and it returns a buffer that can be written to.
pickle.dump(preprocessing, artifact_output_version)
output_artifact.save()


output_artifact = session.io.output('Trained_Logistic_Regression') #use alias as determined per UI to get the correct output artifact.
artifact_output_version = output_artifact.file("trained_model.pickle",ProviderFactoryFilesystem()) #.file() takes a name and a provider and it returns a buffer that can be written to.
pickle.dump(log_reg, artifact_output_version)
output_artifact.save()

